In [ ]:
try:
  # This command only in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Get project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

!unzip cats_and_dogs.zip

PATH = 'cats_and_dogs'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Get number of files in each directory. The train and validation directories
# each have the subdirecories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
# 3

# Create ImageDataGenerator instances for train, validation, and test datasets.
# Rescale pixel values from [0, 255] → [0, 1].
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

# Generate batches of image data (and labels) directly from directory structures.
train_data_gen = train_image_generator.flow_from_directory(
    directory=train_dir,
    batch_size=batch_size,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'   # two classes: cats and dogs
)

val_data_gen = validation_image_generator.flow_from_directory(
    directory=validation_dir,
    batch_size=batch_size,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

# For the test set: there are no subfolders (no labels).
# We use flow_from_directory trick by pointing to a parent folder and
# passing shuffle=False to preserve order.
test_data_gen = test_image_generator.flow_from_directory(
    directory=PATH,                # parent folder that contains 'test'
    classes=['test'],              # explicitly choose only 'test' subfolder
    batch_size=batch_size,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode=None,               # no labels for test data
    shuffle=False
)


In [ ]:
# Define a function to visualize a batch of images
def plotImages(images_arr, probabilities = False):
    # Create a figure with one subplot per image
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5, len(images_arr) * 3))

    # If no probabilities provided, just display images
    if probabilities is False:
        for img, ax in zip(images_arr, axes):
            ax.imshow(img)
            ax.axis('off')
    else:
        # Display images with probability titles
        for img, probability, ax in zip(images_arr, probabilities, axes):
            ax.imshow(img)
            ax.axis('off')
            if probability > 0.5:
                ax.set_title("%.2f" % (probability*100) + "% dog")
            else:
                ax.set_title("%.2f" % ((1-probability)*100) + "% cat")
    plt.show()

# Extract one batch of training images and labels from the generator
sample_training_images, _ = next(train_data_gen)

# Plot first five images from that batch
plotImages(sample_training_images[:5])


In [ ]:
# 5

train_image_generator = ImageDataGenerator(
    rescale=1./255,            # (1) normalization — always include this
    rotation_range=40,         # (2) random rotations up to 40 degrees
    width_shift_range=0.2,     # (3) horizontal shifts
    height_shift_range=0.2,    # (4) vertical shifts
    shear_range=0.2,           # (5) shear transformations
    zoom_range=0.2,            # (6) random zoom in/out
    horizontal_flip=True,      # (7) flip images horizontally
    fill_mode='nearest'        # handles empty pixels created by transformations
)




In [ ]:
# 6
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

In [ ]:
# 7

model = Sequential([

    # --- Convolution + Pooling Layers ---
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    # --- Flatten + Dense Layers ---
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),                   # helps reduce overfitting
    Dense(1, activation='sigmoid')  # binary classification (cat vs. dog)
])

# --- Compile the Model ---
model.compile(
    optimizer='adam',                     # good general-purpose optimizer
    loss='binary_crossentropy',           # binary classification loss
    metrics=['accuracy']                  # track accuracy during training
)

# --- Show Model Summary ---
model.summary()


In [ ]:
# 8

# Train the model using the fit method
history = model.fit(
    x=train_data_gen,                        # training data generator
    steps_per_epoch=total_train // batch_size,  # number of batches per epoch
    epochs=epochs,                           # how many times to go through the data
    validation_data=val_data_gen,            # validation data generator
    validation_steps=total_val // batch_size # number of validation batches per epoch
)


In [ ]:
# 9
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# 10

# Use the trained model to predict probabilities on the test dataset
probabilities = model.predict(test_data_gen)

# Visualize the test images with their predicted probabilities
plotImages(test_data_gen[0][0][:50], probabilities[:50])


In [ ]:
# 11
answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability[0]) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")